# ML Models

In [2]:
import pandas as pd
import numpy as np
import pickle as pkl

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

#### Helper functions

In [3]:
def get_metrics(y_test, y_pred, return_=False):
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    residuals = y_test - y_pred
    res_mean = np.mean(residuals)
    res_std = np.std(residuals)

    print(f'R-squared: {round(r2,3)}')
    print(f'Mean Absolute Error: {"{:,.2f}".format(mae)}')
    print(f'Residuals Mean: {"{:,.2f}".format(res_mean)}, Residuals Std: {"{:,.2f}".format(res_std)}')

    if return_:
        return r2, mae, res_mean, res_std

def save_model(model, filename, path):
  SAVE_PATH = path + filename + '.pkl'
  with open(SAVE_PATH, 'wb') as f:
    pkl.dump(model, f)
    f.close()
  return 'model saved.'

#### Load & Process data

In [9]:
MODEL_SAVE_PATH = r'C:/Users/Nick/Documents/Data Science/Personal projects/car-price-prediction/models/'

DATA_PATH = r'C:/Users/Nick/Documents/Data Science/Personal projects/car-price-prediction/data/clean/'
FILE_NAME = 'data_clean_20240811.csv'

data = pd.read_csv(DATA_PATH+FILE_NAME, sep=';')

data.drop(['FullName', 'ProductionDate'], axis=1, inplace=True)

In [11]:
data_processed = pd.get_dummies(data, prefix_sep = '_')

X = data_processed.drop('Price', axis=1)
y = data_processed['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=123)

## Models building

[*State of Competitive ML report 2023*](https://mlcontests.com/state-of-competitive-machine-learning-2023/)

### LightGBM [TO DO]

#### XGBoost [TO DO]

#### CatBoost [TO DO]

#### SHAP values [TO DO]